<img src="https://drive.google.com/uc?id=1-cL5eOpEsbuIEkvwW2KnpXC12-PAbamr" style="Width:1000px">

# Preprocessing Core Data from IODP


🎯 This exercise will take you through the preprocessing workflow with only a little less help then before. Step by step, feature by feature, you will investigate the dataset and take preprocessing decisions accordingly.

# Introduction to Core Data

![glomar](http://deepseadrilling.org/images/challenger1.jpg)<br>
One of the objectives of this module is to familiarize yourself with new data types. Geologist learn a lot about the history of the Earth by studying core material, i.e. cylindrical pieces of rock recovered from the subsurface by a drill string. Drilling for cores can be performed on land, or at sea from a drilling vessel. One famous drilling vessel is the R/V JOIDES Resolution. I also used to work as a staff scientist on the JOIDES Resolution (aka "JR") before joining Imperial College, so I know it well. The JR is operated by the <a href="https://www.iodp.org/">International Ocean Discovery Program (IODP, 2003-today)</a>, which was preceded by the <a href="http://www-odp.tamu.edu/">Ocean Drilling Program (ODP, 1985-2003)</a> and the <a href="http://deepseadrilling.org/about.htm">Deep Sea Drilling Project (DSDP, 1966-1985)</a>. The various version of the ocean drilling program thus represents one of the major investment in Earth Science over the last 55 years, and has acquired a treasure trove of core and logging data. We will take full advantage of this here.<br>

![cores](https://www.iodp.org/images/right_side_random_images/357_mircobio_carolcotterill.jpg)<br>

Because we will be using ODP/IODP data extensively in this course it is worth defining what the different terms represent.

'Leg' refers to the particular expedition the drill ship was on. In more recent IODP cruises, this is referred to as the 'Expedition' or 'Exp'. Think of a leg as a 6-9 week-long cruise with a single crew, addressing a single research objective. Leg or Expedition are represented by a sequential <code>Int</code>.

'Site' means the name of the general area that was drilled, which is also a sequential number. At each site, the vessel can drill multiple wells, known a 'Hole' and often represented with the letter 'H'. The wells are represented by an ordered <code>str</code>: 'A', 'B', 'C', etc..

Although a drilling vessel aims to recover a continuous section of rock, for practical reasons this is not possible. Thus, pipes are added to the drill string as coring down continues, and a length of rock nominally the length of the drilling pipe is recovered. The pipes on the JR are 9.8 meters, and so we recover cores of 9.8 meters. Cores are denoted with a unique number ('Cor'), followed by the tool used for coring ('T'), which is a 1 character designation of the drilling technique used (more about this later in the course). 'Sc' designates the section: each 9.8 meters-long core is cut into 1.5 meters sections for easy handling, with the addition of the core catcher 'CC' which represents material not in the core liner (in oder words, we are not sure where this material comes from - the core itself, the side of the hole, or the seafloor). For each sample from the ocean drilling program, a 'Top(cm)' is defined and represents the cm down from the top of the section where the sample is located.

Thus, an ODP/IODP sample can be fully spatially resolved by the combination of the parameters we have described above (and that are always supplied as part of the dataset as a unique identifyer). For instance, an example of a sample would be ODP Sample **198-1207-A-2-H-2-65**. This means that this particular sample comes from ODP Leg 198, Site 1207, Hole A (or 'well A'), core 2 drilling with a piston corer ('H'), section 2, 65 cm down section. With this information the 'Depth (mbsf)' which is the depths below seafloor of any sample can be calculated.

Multiple sets of properties can be measured on a core sample, either onboard the ship or postcruise, once the scientists are back onshore. All of this information can be precisely correlated and used by data scientists like yourselves.

![cores](https://iodp.tamu.edu/database/wholecore.jpg)

👇 Run the code below to load the dataset and features you will be working with.

In [ ]:
from nbta.utils import download_data
download_data(id='1l-8v0bV_qY8OSoc1QwcflQyKHKUPHK6y')

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('raw_data/core_data.csv')
data

👉 Take the time to do a preliminary investigation of the features. Notice that you have a column names 'NGR total counts (cps)': this represents the natural gamma ray reading of the core, and there is a complete explanation available [here](http://www-odp.tamu.edu/publications/tnotes/tn26/CHAP5.PDF). You will also see columns named 'L*', 'a*', and 'b*': this refers to the reflectance data from the core, expressed in the CIELAB color space. You can learn much more about this by reading [this techincal note](http://www-odp.tamu.edu/publications/tnotes/tn26/CHAP7.PDF). Reading the technical notes is not essential to understanding this exercise, so consider this optional but good to have.

# Duplicates

ℹ️ Duplicates in datasets can cause data leakage. It is important to locate and remove any meaningless duplicates.

❓ How many duplicated rows are there in the dataset? Save your answer under variable name `duplicate_count`.

In [ ]:
duplicate_count = data.duplicated().sum()
duplicate_count

👇 Remove the duplicates from the dataset. Overwite the dataframe `data`.

In [ ]:
data = data.drop_duplicates()
len(data)

# Splitting your data

Now split your dataset into a `train_set` (80%) and a `test_set` (20%). Use `42` as your `random_state`.

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, train_size=.8, random_state=42)

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('duplicates',
                         duplicates = duplicate_count,
                         dataset = data
)

result.write()
print(result.check())

# Missing data
From now on, use <span style="color:red">only your `train_set`</span>, ***NOT*** your `data` or your `test_set`.

👇 Print out the percentage of missing values for all columns of the dataframe.

In [ ]:
train_set.isnull().sum()/len(data)

## `Lithology`

👇 Investigate the missing values in `Lithology`. Then, chose one of the following solutions:

1. Drop the column entirely
2. Impute the column median using Sklearn's `SimpleImputer`
3. Preserve the NaNs and replace by actual meaning

Make changes effective in the dataframe `data`.


<details>
    <summary>💡 Hint</summary>
    ℹ️ <code>Lithology</code> has a lot of missing values. The description does not touch on what they represent. As such, it is better not to make any assumptions and to drop the column entirely.
</details>

In [ ]:
train_set = train_set.drop('Lithology', axis=1)

## `NGR total counts (cps)`

👇 Investigate the missing values in `NGR total counts (cps)`. Then, chose one of the following solutions:

1. Drop the column entirely
2. Impute the column median using Sklearn's `SimpleImputer`
3. Preserve the NaNs and replace by actual meaning

Make changes effective in the dataframe `data`.


<details>
    <summary>💡 Hint</summary>
ℹ️ <code>NGR total counts (cps)</code> has a few missing values that can be imputed by the median value.
</details>

In [ ]:
from sklearn.impute import SimpleImputer

NGRimputer = SimpleImputer(strategy='median')

train_set['NGR total counts (cps)'] = NGRimputer.fit_transform(train_set[['NGR total counts (cps)']])

👇 When you are done, print out the percentage of missing values for the entire dataframe.

In [ ]:
train_set.isnull().sum().sum()/len(data)

⚠️ Be careful: not all missing values are represented `np.nans`, and python's `isnull()` only detects `np.nans` ⚠️

## `Reflectance L*, a* and b*`

👇 Investigate the missing values in `Reflectance L*, a*, b*`. Then, chose one of the following solutions:

1. Drop the column entirely
2. Impute the column mean using Sklearn's `SimpleImputer`
3. Preserve the NaNs and replace by actual meaning

Make changes effective in the dataframe `data`.


<details>
    <summary>💡 Hint</summary>
ℹ️ <code>Reflectance L*, a*, b*</code> have few missing values that can be imputed by the mean value. You can do this with a single imputer.
</details>

In [ ]:
RefImputer = SimpleImputer(strategy='mean')
train_set[['Reflectance L*', 'Reflectance a*','Reflectance b*']]=RefImputer.fit_transform(train_set[['Reflectance L*','Reflectance a*','Reflectance b*']])

## `Type`

👇 Investigate the missing values in `Type`. Then, chose one of the following solutions:

1. Drop the column entirely
2. Impute the column median
3. Preserve the NaNs and replace by actual meaning

Make changes effective in the dataframe `data`.


<details>
    <summary>💡 Hint</summary>
ℹ️ <code>Type</code> represents the type of the coring apparatus used. It is a string, not a number. Only a few values are missing, and you don't want to loose important information by dropping the rows: instead you could assume that the type can be replaced by the most frequent value without loosing much information. Check the <code>SimpleImputer</code> documentation to see how to do that. 
</details>

In [ ]:
typeImputer = SimpleImputer(strategy='most_frequent')
train_set[['Type']] = typeImputer.fit_transform(train_set[['Type']])

## `Depth CSF-A (m)`

👇 Investigate the missing values in `Depth CSF-A (m)`. Remember that this represents the depth of the sample below surface, a critical piece of information that cannot easily be supplemented by other data. With this in mind, chose one of the following solutions:

1. Drop the rows entirely
2. Impute the column median
3. Preserve the NaNs and replace by actual meaning

Make changes effective in the dataframe `data`.


<details>
    <summary>💡 Hint</summary>
ℹ️ <code>Depth CSF-A (m)</code> is only missing in a few rows. Because we cannot supplement it easily and because a 'mean' depth for a sample has no real meaning, a good strategy here would be to drop the rows where this value is NaN. Look at the pandas documentation to see how to drop specific rows: think also of how you would find the index of the rows that need dropping.
</details>

In [ ]:
index = train_set[train_set['Depth CSF-A (m)'].isnull() == True].index

train_set.drop(index,inplace=True)

### Check to see if you still have missing values

In [ ]:
train_set.isnull().sum()

In [ ]:
len(train_set.columns)

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('missing_values',
                         dataset = train_set
)

result.write()
print(result.check())

# Scaling
In order to investigate features and, we recommend that you plot a histogram and a box plot for each one of the features.

##  `Reflectance L*` 

👇 Investigate `Reflectance L*` for distribution and outliers. Then, choose the most appropriate scaling technique. Either:

1. Standard Scale
2. Robust Scale
3. MinMax Scale

Replace the original columns by the transformed values.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(train_set['Reflectance L*']);

In [ ]:
train_set[['Reflectance L*']].boxplot();

<details>
    <summary>💡 Hint</summary>
ℹ️ Since <code>Reflectance L*</code> does  seem to have a normal distribution, so we can use the <code>StandardScaler</code> to scale. Note that it would not be incorrect to use <code>MinMax()</code> or <code>RobustScaler()</code>.
</details>

In [ ]:
from sklearn.preprocessing import StandardScaler

train_set['Reflectance L*'] = StandardScaler().fit_transform(train_set[['Reflectance L*']])

## `Refectance a*` & `Reflectance b*`

👇 Investigate `Reflectance a*` & `Reflectance b*`. Then, chose one of the following scaling techniques:

1. MinMax Scale
2. Standard Scale
3. Robust Scale

Replace the original columns by the transformed values.

<details>
    <summary>💡 Hint</summary>
ℹ️ <code>Reflectance a*</code> and <code>Reflectance b*</code> are both normally distributed but with some outliers: we could use the <code>RobustScaler()</code> here.
</details>

In [ ]:
import matplotlib.pyplot as plt

plt.hist(train_set['Reflectance a*']);

In [ ]:
train_set[['Reflectance a*']].boxplot()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(train_set['Reflectance b*']);

In [ ]:
train_set[['Reflectance b*']].boxplot()

In [ ]:
from sklearn.preprocessing import RobustScaler

abScaler = RobustScaler()
train_set[['Reflectance a*','Reflectance b*']] = abScaler.fit_transform(train_set[['Reflectance a*','Reflectance b*']]) 

## `Depth CSF-A (m)`

👇 Investigate `Depth CSF-A (m)` for distribution and outliers. Then, choose the most appropriate scaling technique. Either:

1. Standard Scale
2. Robust Scale
3. MinMax Scale

Replace the original columns by the transformed values.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(train_set['Depth CSF-A (m)']);

In [ ]:
train_set[['Depth CSF-A (m)']].boxplot();

<details>
    <summary>💡 Hint</summary>
ℹ️ <code>Depth CSF-A (m)</code> does not show a normal distribution, it is better to <code>MinMax()</code> scale to ensure all results are between 0 and 1.
</details>

In [ ]:
from sklearn.preprocessing import MinMaxScaler

depthScaler = MinMaxScaler()

train_set['Depth CSF-A (m)'] = depthScaler.fit_transform(train_set[['Depth CSF-A (m)']])

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('scaling',
                         dataset = train_set
)

result.write()
print(result.check())

# Feature Engineering

## `Type`

👇 Investigate `Type` and chose one of the following encoding techniques accordingly:
- Ordinal encoding
- One-Hot encoding

Add the encoding to the dataframe as new colum(s), and remove the original column.


<details>
    <summary>💡 Hint</summary>
ℹ️ `Type` is a multicategorical feature that must be One hot encoded: there is no explicit ordinal value to the drill bit type.
</details>

In [ ]:
# Check what unique value of types exist:
train_set.Type.unique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False).fit(train_set[['Type']])

In [ ]:
ohe.categories_

In [ ]:
train_set['H'],train_set['X'],train_set['R']= ohe.transform(train_set[['Type']]).T # We need to transpose the data into columns

train_set.drop('Type', axis=1, inplace = True)

train_set

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('encoding', dataset = train_set, new_features = ['H', 'X', 'R'])

result.write()
print(result.check())

# 🏁